In [ ]:
# Import libraries.
import tensorflow as tf
from tensorflow import keras
import IPython
!pip install -q -U keras-tuner
import kerastuner as kt

In [ ]:
# Load the dataset and split it into train and test.
# It is important to note that we have a dataset of 28x28 images, since we will have to take account of this afterwards, especially at the time
# of passing the data onto our model.
fashion_mnist = keras.datasets.fashion_mnist 
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()  

In [ ]:
# Due to the fact that the images contain pixel values between 0 and 255, it would be great if we make the pixel values go from 0 to 1,
# so it is easier to compute.
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
# Then, we can build our model. In this case, it is a sequential one, consisting of 1 input layer, 1 hidden layer and 1 output layer.
# As we can see, it contains several parameters that are already set, like the numbers of neurons of each layer, the optimizer, the loss function, etc.
# Finally, it is well noting that since we intend to classify 10 things, the last layer should contain 10 nodes, as stated below, meaning that
# for our project, since we intend to classify 5 movements, that layer should contain 5 nodes instead of 10.
model = keras.Sequential([
  keras.layers.Flatten(input_shape=(28, 28)),
  keras.layers.Dense(256, activation='relu'), 
  keras.layers.Dense(10, activation='softmax') 
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Now, once our model is defined, we can start training it by using the data we have already processed.
model.fit(train_images, train_labels, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4831 - accuracy: 0.8291
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3643 - accuracy: 0.8667
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3275 - accuracy: 0.8803
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3024 - accuracy: 0.8873
Epoch 5/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2844 - accuracy: 0.8952
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2676 - accuracy: 0.9007
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2564 - accuracy: 0.9048
Epoch 8/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2443 - accuracy: 0.9091
Epoch 9/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2363 - accuracy: 0.9113
Epoch 10/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.227

In [ ]:
# After training, it comes the test process, meaning that once our model learned, it is time for it to recognize unseen data.
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=1) 
print('Test accuracy:', test_acc)

313/313 [==============================] - 1s 2ms/step - loss: 0.3373 - accuracy: 0.8852
Test accuracy: 0.885200023651123


In [ ]:
# As we can see, the testing accuracy is lower than the training one. This is because we previously overfitted our model.
# For that purpose, we can choose the parameters which best fit our model (hyptertuning).
# For this purpose, we will use the Keras Tuner.

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Choose the best number of units on the first dense layer.
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Choose a proper learning rate. In this case, we will choose between 0.01, 0.001, or 0.0001.
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

  return model

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')    

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


In [ ]:
# We also need to define a callback in order to clear the training outputs in every training step.
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
# Get the optimal hyperparameters
tuner.search(train_images, train_labels, epochs = 10, validation_data = (test_images, test_labels), callbacks = [ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 384 and the optimal learning rate for the optimizer
is 0.001.



In [ ]:
# Build the model with the best parameters and train the model.
model = tuner.hypermodel.build(best_hps)
model.fit(train_images, train_labels, epochs = 10, validation_data = (test_images, test_labels))

Epoch 1/10
1875/1875 [==============================] - 12s 7ms/step - loss: 0.4783 - accuracy: 0.8311 - val_loss: 0.4197 - val_accuracy: 0.8518
Epoch 2/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3614 - accuracy: 0.8683 - val_loss: 0.3717 - val_accuracy: 0.8654
Epoch 3/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3251 - accuracy: 0.8792 - val_loss: 0.3628 - val_accuracy: 0.8648
Epoch 4/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.3003 - accuracy: 0.8892 - val_loss: 0.3626 - val_accuracy: 0.8696
Epoch 5/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2836 - accuracy: 0.8943 - val_loss: 0.3448 - val_accuracy: 0.8746
Epoch 6/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2653 - accuracy: 0.9012 - val_loss: 0.3621 - val_accuracy: 0.8662
Epoch 7/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2552 - accuracy: 0.9041 - val_loss: 0.3629 - val_ac

In [ ]:
# Test the model
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=1) 
print('Test accuracy:', test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.3404 - accuracy: 0.8855
Test accuracy: 0.8855000138282776


In [ ]:
# Finally, we see how there is no difference between the training accuracy and the test one, so we can say that we got rid of the initial overfitting.